In [15]:
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
import gym
import numpy as np
import sys
sys.path.append("../../")
from Hack import load, rl
%matplotlib qt5

In [16]:
epex = load.epex().load()
price_array = epex['apx_da_hourly'].values

In [17]:
max_time = 30769 
#! This is a horrible way of doing this, but gets the last date in 2020
# env = helpers.energy_price_env(price_array, max_time=max_time, window_size=24*2)
# model = PPO(MlpPolicy, env, verbose=1)
# check_env(env, warn=True)

In [18]:
# mean_reward_before_train = helpers.evaluate(model, num_episodes=1, index = epex.index)

In [19]:
from typing import Callable
from stable_baselines3.common.utils import set_random_seed

def make_env(rank: int, seed: int = 0) -> Callable:
    """
    Utility function for multiprocessed env.
    
    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environment you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    :return: (Callable)
    """
    def _init() -> gym.Env:
        env = rl.energy_price_env(price_array, max_time=max_time, window_size=24*2)
        env.seed(seed + rank)
        return env
    set_random_seed(seed)
    return _init

In [20]:
from stable_baselines3.common.vec_env import SubprocVecEnv
num_cpu = 4
env = SubprocVecEnv([make_env(i) for i in range(num_cpu)])
model = PPO(MlpPolicy, env, verbose = 0)
#model.learn(total_timesteps = 10000)

In [21]:
# # Train the agent for 10000 steps
# model.learn(total_timesteps=10000)
model.learn(total_timesteps = 25000)
model.save("../model_dir/trained_model_multiprocess")

In [22]:
# # Trained Agent, after training
# periods = 4 * 28*24*2 # len(price_array)-1  # 4 * 28*24*2
# new_env =  DummyVecEnv([lambda: helpers.energy_price_env(price_array, start_time=max_time, max_time = periods)])
# mean_reward_after_train = helpers.evaluate(loaded_model, new_env=new_env, num_episodes=5, index=epex.index)